In [1]:
#Importando librerías.
import pandas as pd
import sqlite3

En esta parte del programa trata de importar un archivo .csv a un manejable con sql.

In [2]:
#Crea mi dataframa porveninte de un archivo csv.
file = 'lluvia_2019.csv'
df = pd.read_csv(file)

In [3]:
#Compruebo si se ha cargado bien.
df.head()

,Unnamed: 0,LON,LAT,EDO,CLAVE_SIH,NOMBRE,ENE
0,0,-102.309722,21.895000,AGS,AGSAG,"Aguascalientes, Ags.",6.61
1,1,-102.466944,22.188611,AGS,ANVAG,"Cincuenta Aniversario, Ags.",10.60
2,2,-102.184167,21.738611,AGS,BRTAG,"San Bartolo, Ags.",10.50
3,3,-102.712222,21.849167,AGS,CALVILLO,"Calvillo, Ags. SMN*",18.17
4,4,-102.676944,21.997500,AGS,CDRAG,"La Codorniz, Ags.",16.70


In [4]:
#Reviso el nombre de las columnas.
df.columns

Index(['Unnamed: 0', 'LON', 'LAT', 'EDO', 'CLAVE_SIH', 'NOMBRE', 'ENE'], dtype='object')

In [5]:
#Elimino la primera columna que es irrelevante.
df=df.drop(['Unnamed: 0'], axis=1)

In [6]:
#Creo la conexión a la base de datos 'ejemplo.db'(si no existe, la crea)
conn = sqlite3.connect("ejmplo.db")

#cerramos la conexión
conn.close()

In [7]:
conn=sqlite3.connect("ejmplo.db") 
#guardo la tabla en ejmplo.db y le pongo el nombre de lluvia2019, si ya existe la reemplaza

df.to_sql("lluvia2019", conn, if_exists="replace")

conn.close()

In [8]:
#vamos a tratar de manejar la base de datos
conn = sqlite3.connect("ejmplo.db")
cur = conn.cursor()

cur.execute("select * from lluvia2019 limit 5;")
results = cur.fetchall()
print(results)

cur.close()
conn.close()

[(0, -102.309722, 21.895, 'AGS', 'AGSAG', 'Aguascalientes, Ags.', 6.61), (1, -102.466944, 22.188611, 'AGS', 'ANVAG', 'Cincuenta Aniversario, Ags.', 10.6), (2, -102.184167, 21.738611, 'AGS', 'BRTAG', 'San Bartolo, Ags.', 10.5), (3, -102.712222, 21.849167, 'AGS', 'CALVILLO', 'Calvillo, Ags. SMN*', 18.17), (4, -102.676944, 21.9975, 'AGS', 'CDRAG', 'La Codorniz, Ags.', 16.7)]


vamos a leer ahora el archivo con pandas proveniente de la base de datos, aunque ya existe el dataframe lo hago para seguir el tutorial.

In [9]:
conn = sqlite3.connect("ejmplo.db")
df2 = pd.read_sql_query("select * from lluvia2019 limit 5;", conn)

conn.close()

In [10]:
df2

,index,LON,LAT,EDO,CLAVE_SIH,NOMBRE,ENE
0,0,-102.309722,21.895000,AGS,AGSAG,"Aguascalientes, Ags.",6.61
1,1,-102.466944,22.188611,AGS,ANVAG,"Cincuenta Aniversario, Ags.",10.60
2,2,-102.184167,21.738611,AGS,BRTAG,"San Bartolo, Ags.",10.50
3,3,-102.712222,21.849167,AGS,CALVILLO,"Calvillo, Ags. SMN*",18.17
4,4,-102.676944,21.997500,AGS,CDRAG,"La Codorniz, Ags.",16.70


In [11]:
df2['NOMBRE']

0           Aguascalientes, Ags.
1    Cincuenta Aniversario, Ags.
2              San Bartolo, Ags.
3            Calvillo, Ags. SMN*
4              La Codorniz, Ags.
Name: NOMBRE, dtype: object

In [13]:
conn = sqlite3.connect("ejmplo.db")
cur = conn.cursor()

#insertamos un renglón con valores arbitrarios
cur.execute("insert into lluvia2019 values (1941,-100,20,'GTO','GTOJR','Juventino Rosas, Gto.',5.00)")

#es necesario este comando para llevar a cabo las modificaciones
conn.commit()
a=pd.read_sql_query("select * from lluvia2019 where NOMBRE='Juventino Rosas, Gto.';", conn)
cur.close()
conn.close()

In [14]:
#Comprobamos que este dentro del database
a

,index,LON,LAT,EDO,CLAVE_SIH,NOMBRE,ENE
0,1941,-100.0,20.0,GTO,GTOJR,"Juventino Rosas, Gto.",5.0


# Updating

In [18]:
#Updating
conn = sqlite3.connect("ejmplo.db")
cur = conn.cursor()

values = (20.64, 'GTOJR')
cur.execute("update lluvia2019 set LAT=? where CLAVE_SIH=?", values)

conn.commit()
a=pd.read_sql_query("select * from lluvia2019 where CLAVE_SIH='GTOJR';", conn)
cur.close()
conn.close()
a

,index,LON,LAT,EDO,CLAVE_SIH,NOMBRE,ENE
0,1941,-100.0,20.64,GTO,GTOJR,"Juventino Rosas, Gto.",5.0


# Deleting

In [19]:
conn = sqlite3.connect("ejmplo.db")
cur = conn.cursor()

values=('GTOJR',)
cur.execute("delete from lluvia2019 where CLAVE_SIH=?", values)
conn.commit
a=pd.read_sql_query("select * from lluvia2019 where CLAVE_SIH='GTOJR';", conn)

cur.close()
conn.close()
a

,index,LON,LAT,EDO,CLAVE_SIH,NOMBRE,ENE
